In [ ]:
import pandas as pd

df = pd.read_csv('../Apple462.csv')
df = df.drop(df[df["region"] != "UK"].index)
df = df.drop(columns=["Unnamed: 0"])
df

In [ ]:
df_envi = df.drop(columns=["Fuji", "Gala", "region"])
df_envi

# print(df_envi['Envi'])

# df_envi.to_csv("envi.csv", sep=",", encoding="utf-8", index=False)